In [9]:
from pymongo import MongoClient

In [10]:
client = MongoClient("mongodb://mongodb:27017/")

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("myApp") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.2') \
.getOrCreate()

In [12]:
h1n1 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
"mongodb://mongodb:27017/epidemics.h1n1_days").load()

In [13]:
h1n1.createOrReplaceTempView("h1n1")

In [15]:
h1n1 = spark.sql("SELECT _id, country, date_format(update_time, 'YYYY-MM-dd') as date, \
COALESCE(int((cases - lag(cases,1) over (partition by country ORDER BY update_time asc))), cases) as cases, cases as cumul_cases, \
COALESCE(int((deaths - lag(deaths,1) over (partition by country ORDER BY update_time asc))), deaths) as deaths, deaths as cumul_deaths \
FROM h1n1 WHERE country <> 'Grand Total'")

In [16]:
h1n1.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://mongodb:27017/epidemics2.epidemics_h1n1").save()